<a href="https://colab.research.google.com/github/talhaanwarch/Sarcasm-nlp/blob/master/reddit_multi_input_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub
!pip install tensorflow-addons
#!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 27.4MB/s 
     |████████████████████████████████| 450kB 40.7MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 1.0MB 6.5MB/s 
     |████████████████████████████████| 1.0MB 6.7MB/s 


In [3]:
cd /content/drive/My\ Drive/dataset/sarcasm

/content/drive/My Drive/dataset/sarcasm


In [4]:
ls

model_gru.png
__pycache__/
sarcasm_detection_shared_task_reddit_testing.jsonl
sarcasm_detection_shared_task_reddit_training.jsonl
sarcasm_detection_shared_task_twitter_testing.jsonl
sarcasm_detection_shared_task_twitter_training.jsonl
tokenization.py


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score


In [0]:
reddit_train=pd.read_json('sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)

In [10]:
reddit_train.head()

,label,response,context
0,SARCASM,"Yeah I mean there's only one gender anyways, w...",[LPT: If you're worried about hurting someone'...
1,SARCASM,"Sounds like you don't like science, you theist...",[Promotional images for some guy's Facebook pa...
2,SARCASM,"Ofc play them in try mode, Blizzard were so ge...",[My friends won't play Dota2; I won't play LoL...
3,SARCASM,"I don't understand, Reddit told me that Hillar...",[Poll: Convention boosts Clinton to 11-point l...
4,SARCASM,"yeh, they're the reigning triple premiers, why...",[Wayne Ludbey: Jordan Lewis has the ultimate c...


In [0]:
#reddit_train['response']=reddit_train['response'].str.replace('@USER', "") 
reddit_train['response']=reddit_train['response'].str.replace('\d+', '')
reddit_train['response']=reddit_train['response'].str.lower()
reddit_train['response']=reddit_train['response'].str.replace('[^\w\s]','')


In [12]:
reddit_train.head()

,label,response,context
0,SARCASM,yeah i mean theres only one gender anyways wom...,[LPT: If you're worried about hurting someone'...
1,SARCASM,sounds like you dont like science you theist s...,[Promotional images for some guy's Facebook pa...
2,SARCASM,ofc play them in try mode blizzard were so gen...,[My friends won't play Dota2; I won't play LoL...
3,SARCASM,i dont understand reddit told me that hillary ...,[Poll: Convention boosts Clinton to 11-point l...
4,SARCASM,yeh theyre the reigning triple premiers why br...,[Wayne Ludbey: Jordan Lewis has the ultimate c...


In [0]:
reddit_train['context']=reddit_train['context'].apply(lambda x: ','.join(map(str, x)))
#reddit_train['context']=reddit_train['context'].str.replace('@USER', "") 
reddit_train['context']=reddit_train['context'].str.lower()
reddit_train['context']=reddit_train['context'].str.replace('[^\w\s]','')
reddit_train['context']=reddit_train['context'].str.replace('\d+', '')


In [14]:
reddit_train.head()

,label,response,context
0,SARCASM,yeah i mean theres only one gender anyways wom...,lpt if youre worried about hurting someones fe...
1,SARCASM,sounds like you dont like science you theist s...,promotional images for some guys facebook page...
2,SARCASM,ofc play them in try mode blizzard were so gen...,my friends wont play dota i wont play lol is l...
3,SARCASM,i dont understand reddit told me that hillary ...,poll convention boosts clinton to point lead o...
4,SARCASM,yeh theyre the reigning triple premiers why br...,wayne ludbey jordan lewis has the ultimate com...


In [15]:
len(reddit_train)

4400

In [16]:
from collections import Counter
Counter(reddit_train['label'])

Counter({'NOT_SARCASM': 2200, 'SARCASM': 2200})

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(reddit_train['label'])

In [18]:
reddit_test=pd.read_json('sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)
reddit_test.head()

,context,response,id
0,[Dear Trump: the bill is now on your desk. Vet...,"I was elected to golf not to uh, got nothing.",reddit_1
1,"[Thailand cave rescue: All 12 boys, coach free...",I thought those kids were in a very bad spot. ...,reddit_2
2,"[Snapchat: ""To attract more Android users, we ...","Nothing gives off that hipster, low budget, st...",reddit_3
3,[Nickelodeon just uploaded a high quality vers...,A major corporation would run a kickstarter so...,reddit_4
4,[Before you all start theorizing about Mats re...,"Yup, scott “accidentally” added a last name to...",reddit_5


In [0]:
#reddit_test['response']=reddit_test['response'].str.replace('@USER', "") 
reddit_test['response']=reddit_test['response'].str.replace('\d+', '')
reddit_test['response']=reddit_test['response'].str.lower()
reddit_test['response']=reddit_test['response'].str.replace('[^\w\s]','')
reddit_test['context']=reddit_test['context'].apply(lambda x: ','.join(map(str, x)))
#reddit_test['context']=reddit_test['context'].str.replace('@USER', "") 
reddit_test['context']=reddit_test['context'].str.lower()
reddit_test['context']=reddit_test['context'].str.replace('[^\w\s]','')
reddit_test['context']=reddit_test['context'].str.replace('\d+', '')


In [21]:
reddit_test.head()

,context,response,id
0,dear trump the bill is now on your desk veto t...,i was elected to golf not to uh got nothing,reddit_1
1,thailand cave rescue all boys coach freed lat...,i thought those kids were in a very bad spot t...,reddit_2
2,snapchat to attract more android users we are ...,nothing gives off that hipster low budget star...,reddit_3
3,nickelodeon just uploaded a high quality versi...,a major corporation would run a kickstarter so...,reddit_4
4,before you all start theorizing about mats rec...,yup scott accidentally added a last name to a ...,reddit_5


#BERT

In [0]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
train_response = bert_encode(reddit_train.response, tokenizer, max_len=200)
train_context = bert_encode(reddit_train.context, tokenizer, max_len=200)
train_labels=reddit_train.label.values
#test_input = bert_encode(test_bert.values, tokenizer, max_len=160)

In [0]:
test_response = bert_encode(reddit_test.response, tokenizer, max_len=200)
test_context = bert_encode(reddit_test.context, tokenizer, max_len=200)

In [0]:
test_gen=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

In [0]:
 max_len=200

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa

In [0]:
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)

# ...

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

In [0]:
contex_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_word_ids")
contex_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_mask")
contex_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_segment_ids")
_, context_sequence_output = bert_layer([contex_input_word_ids, contex_input_mask, contex_segment_ids])
context_clf_output = context_sequence_output[:, 0, :]


reponse_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_word_ids")
reponse_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_mask")
reponse_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_segment_ids")
_, reponset_sequence_output = bert_layer([reponse_input_word_ids, reponse_input_mask, reponse_segment_ids])
reponset_clf_output = reponset_sequence_output[:, 0, :]



conc=concatenate([context_clf_output,reponset_clf_output])
conc=Dropout(0.3)(conc)
out = Dense(1, activation='sigmoid')(conc)
    
model = Model(inputs=[contex_input_word_ids, contex_input_mask, contex_segment_ids,reponse_input_word_ids,reponse_input_mask,reponse_segment_ids], outputs=out)
model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    


In [0]:
train_gen=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

In [33]:
len(train_gen),len(train_label)

(6, 4400)

In [34]:
train_context[0].shape

(4400, 200)

In [0]:
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold


In [0]:
def unpack(a, b, c, d,e,f,g): 
    #print(a[g].shape, b[g].shape, c[g].shape, d[g].shape,e[g].shape,f[g].shape) 
    return [a[g], b[g], c[g], d[g],e[g],f[g]]

In [0]:
#unpack(*train_gen,train)

In [0]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=False)
scores = []
test=[]
s=0
for train, val in kfold.split(list(zip(*train_gen)), train_label):
  model.fit(unpack(*train_gen,train),train_label[train],batch_size=4,epochs=5,verbose=2,
            validation_data=(unpack(*train_gen,val), train_label[val]))
   

  y_pred = model.predict(unpack(*train_gen,val), batch_size=4, verbose=1)

  test.append(model.predict(test_gen, batch_size=4, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(train_label[val], y_pred, average='macro')
  s+=1
  print(s,' f1 score  is ',f1)
  scores.append(f1)
  tf.keras.backend.clear_session()

Train on 3520 samples, validate on 880 samples
Epoch 1/5
